# Hyperparameters

In [1]:
hparam = {
    'epoch': 100,
    'hid_dim': 10,
    'gru_layers': 2,
    'lr': 0.0001,
    'report': 1
}

# Configure Dataset

In [2]:
from os import listdir
from itertools import chain
from random import shuffle
import pandas as pd

# Load tensor file(pickle) list
dpath = './Dataset/matches_tensor/legacy/'
servers = listdir(dpath)
mat_lists = list(chain(*[listdir(dpath+server) for server in servers])) # total match lists

# Divide dataset into 3 (tr, val, te)
shuffle(mat_lists)
tr_set = mat_lists[:190000]
val_set = mat_lists[190000:200000]
te_set = mat_lists[200000:]

# Load match results as dictionaris
rpath = './Dataset/matches_result/'
mat_results = {}
for server in servers:
    fname = rpath + f'{server}_results.csv'
    mat_results[server] = pd.read_csv(fname, index_col=0).to_dict('index')

{'210513_eun1_2829252949': {'win': 'red', 'duration': 1841248}, '210512_eun1_2827556633': {'win': 'red', 'duration': 1889954}, '210510_eun1_2825588465': {'win': 'red', 'duration': 1258429}, '210513_eun1_2829222106': {'win': 'red', 'duration': 2142010}, '210513_eun1_2829249597': {'win': 'blue', 'duration': 1496969}, '210510_eun1_2825569407': {'win': 'red', 'duration': 1818197}, '210510_eun1_2825476511': {'win': 'red', 'duration': 1767126}, '210510_eun1_2825573092': {'win': 'red', 'duration': 2511506}, '210510_eun1_2825554734': {'win': 'red', 'duration': 1553206}, '210513_eun1_2829181707': {'win': 'blue', 'duration': 2890074}, '210517_eun1_2833096077': {'win': 'blue', 'duration': 1803332}, '210513_eun1_2829183709': {'win': 'blue', 'duration': 1910930}, '210513_eun1_2829248469': {'win': 'blue', 'duration': 1535334}, '210513_eun1_2829236730': {'win': 'blue', 'duration': 1439565}, '210512_eun1_2827511362': {'win': 'blue', 'duration': 2004839}, '210512_eun1_2827538787': {'win': 'red', 'durat

# Model

In [3]:
from model import Model

M = Model(hparam['hid_dim'], hparam['gru_layers'], hparam['lr'])
# M.load_parameters('parameters.pkl') # remove comment if you want load parameter file

# Train

In [4]:
import pickle

for epoch in range(hparam['epoch']):
    # training process
    for idx, tr_dat in enumerate(tr_set):
        for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
            if serv in tr_dat:
                server = serv
                continue
        mat = tr_dat[:-4]

        # print training progress
        end_char = '\n' if idx+1==len(tr_set) else '\r'
        print(f'[Epoch {epoch+1:3d}] ({idx+1:6d}/{len(tr_set)}) processing {mat:<30s}', end=end_char)

        with open(f'{dpath}{server}/{tr_dat}', 'rb') as feat_file:
            features = pickle.load(feat_file)
        winner = mat_results[server][mat]['win']

        # train
        M.train(features, winner)
    
    # validation process
    if (epoch+1)%hparam['report'] == 0: # execute validation in {hparam['report']} epoch interval
        M.reset_result()
        for idx, val_dat in enumerate(val_set):
            for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
                if serv in val_dat:
                    server = serv
                    continue
            mat = val_dat[:-4]

            # print validation progress
            print(f'[Validation for epoch {epoch+1:3d}] ({idx+1:6d}/{len(val_set)}) processing {mat:<30s}', end='\r') 

            with open(f'{dpath}{server}/{val_dat}', 'rb') as feat_file:
                features = pickle.load(feat_file)
            winner = mat_results[server][mat]['win']
            
            # validate
            M.test(features, winner)

        acc, pre, rec, f1, loss = M.get_result()
        print(f'[Validation for epoch {epoch+1:3d}] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / \
            Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')
        M.save_parameters('parameters.pkl')

KeyboardInterrupt: 

# Test

In [ ]:
M.reset_result()
for idx, te_dat in enumerate(te_set):
    for serv in ['kr', 'euw1', 'eun1', 'na1', 'jp1']:
        if serv in te_dat:
            server = serv
            continue
    mat = te_dat[:-4]

    # print training progress
    end_char = '\n' if idx+1==len(te_set) else '\r'
    print(f'[Epoch {epoch+1:3d}] ({idx+1:6d}/{len(te_set)}) processing {mat:<30s}', end=end_char)

    with open(f'{dpath}{server}/{te_dat}', 'rb') as feat_file:
        features = pickle.load(feat_file)
    winner = mat_results[server][mat]['win']

    M.test(features, winner, server, mat)

    acc, pre, rec, f1, loss = M.get_result()
    print(f'[Test result] Accuracy: {acc:5.4f} / Precision: {pre:5.4f} / \
        Recall: {rec:5.4f} / F1 score: {f1:5.4f} / loss: {loss:5.4f}')